# Fine-Tuning DistilBERT for Emotion Classification

**Source:** [dair-ai/emotion Dataset](https://huggingface.co/datasets/dair-ai/emotion) - English Twitter messages with 6 basic emotions: anger, fear, joy, love, sadness, and surprise.

This notebook demonstrates how to fine-tune a DistilBERT model for emotion classification.We preprocess the data, train the model, evaluate it, save it to Google Drive, and provide an interactive demo for predictions.

In [ ]:
!pip install torch torchvision torchaudio -q
!pip install transformers datasets accelerate -q

from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import evaluate # Changed import here
import numpy as np

dataset = load_dataset("dair-ai/emotion")
#connect to drivz
from google.colab import drive
drive.mount('/content/drive')


## Load Dataset and Preprocess


In [ ]:
#tokenization
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# format for PyTorch
tokenized_datasets.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

train_dataset = tokenized_datasets["train"]
val_dataset = tokenized_datasets["validation"]
test_dataset = tokenized_datasets["test"]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

split/train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

split/validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

split/test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

## Load model with correct number of labels

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=6)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Training Configuration

In [ ]:
metric = evaluate.load("accuracy")
#metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

#arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_total_limit=2,
)


#trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-127159963.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


## Train and Evaluate

In [ ]:
trainer.train()
test_results = trainer.evaluate(test_dataset)
print(test_results)


Epoch,Training Loss,Validation Loss,Accuracy
1,0.263700,0.225962,0.921500
2,0.138800,0.158322,0.933500
3,0.105700,0.148673,0.937000


{'eval_loss': 0.16612327098846436, 'eval_accuracy': 0.926, 'eval_runtime': 6.2531, 'eval_samples_per_second': 319.843, 'eval_steps_per_second': 10.075, 'epoch': 3.0}


## Save the model

In [ ]:
save_dir = "/content/drive/MyDrive/distilbert-emotion"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
print("Model saved at", save_dir)

Mounted at /content/drive
Model saved at /content/drive/MyDrive/distilbert-emotion


## Define and test prediction function

In [ ]:
def predict_emotion(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128).to(model.device) # Move inputs to the same device as the model
    outputs = model(**inputs)
    preds = outputs.logits.argmax(dim=1).item()
    label_names = dataset["train"].features["label"].names
    return label_names[preds]

#example
print(predict_emotion("I feel so happy today"))
print(predict_emotion("This is the worst day ever"))

joy
sadness


## Demo Interactive loop

In [ ]:
def predict_emotion(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=128
    ).to(model.device)  # Move inputs to the same device as the model
    outputs = model(**inputs)
    preds = outputs.logits.argmax(dim=1).item()
    label_names = dataset["train"].features["label"].names
    return label_names[preds]

# loop
while True:
    text = input("Enter a sentence (or type 'quit' to exit): ")
    if text.lower() == "quit":
        break
    emotion = predict_emotion(text)
    print(f"Predicted Emotion: {emotion}\n")


Enter a sentence (or type 'quit' to exit): i am extremely joyful today because of my success
Predicted Emotion: joy

Enter a sentence (or type 'quit' to exit): i am so disapointed 
Predicted Emotion: anger

Enter a sentence (or type 'quit' to exit): i will fight you
Predicted Emotion: anger

Enter a sentence (or type 'quit' to exit): i am disgusted
Predicted Emotion: anger

Enter a sentence (or type 'quit' to exit): i am crying
Predicted Emotion: sadness

Enter a sentence (or type 'quit' to exit): i love you
Predicted Emotion: love

Enter a sentence (or type 'quit' to exit): i hate you
Predicted Emotion: anger

Enter a sentence (or type 'quit' to exit): i am scared
Predicted Emotion: fear

Enter a sentence (or type 'quit' to exit): i didnt expect this
Predicted Emotion: anger

Enter a sentence (or type 'quit' to exit): i am surprised
Predicted Emotion: surprise

Enter a sentence (or type 'quit' to exit): quit
